In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [2]:
file_path = 'MOTHER_TABLE.csv'
df = pd.read_csv(file_path).drop('Unnamed: 0', axis=1)

<ipython-input-2-9a78aeb8aeea>:2: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path).drop('Unnamed: 0', axis=1)


In [21]:
def classify_individual(remarks, func_class_1, func_class_2):
    # Handle NaN values
    if pd.isna(remarks):
        remarks = ''
    if pd.isna(func_class_1):
        func_class_1 = ''
    if pd.isna(func_class_2):
        func_class_2 = 0
    
    func_class_1_mapping = {
        101: 'Maritime',
        102: 'Military',
        103: 'Passenger/Civilian'
    }

    func_class_2_mapping = {
        101: 'Leadership',
        102: 'Off. Specialized',
        103: 'Non-mar. Specialized',
        110: 'General',
        111: 'General Spec',
        119: 'Foreign labour force',
        121: 'Other/No info',
        122: 'Other/No info',
        116: 'Trainee Leadership',
        117: 'Trainee Spec',
        118: 'Trainee',
        104: 'Religious',
        105: 'Medical',
        106: 'Servant',
        120: 'Company Admin',
        107: 'Unfree labour',
        108: 'Prisoner',
        109: 'Stowaway',
        112: 'Other/No info'
    }
    
    if func_class_1 in func_class_1_mapping and func_class_2 in func_class_2_mapping:
        return f"{func_class_1_mapping[func_class_1]} {func_class_2_mapping[func_class_2]}"
    
    keywords = {
        'Maritime Leadership': ['capitaine', 'lieutenant', 'enseigne', 'écrivain'],
        'Military': ['soldat', 'sergent', 'caporal', 'cadet'],
        'Servant': ['domestique', 'valet'],
        'Specialized Labour': ['armurier', 'charpentier', 'canonnier'],
        'Passenger': ['passager', 'passagère'],
        'Unfree Labour': ['esclave', 'prisonnier'],
        'Religious': ['missionnaire', 'religieux'],
        'Medical': ['chirurgien', 'apothicaire']
    }
    
    for category, terms in keywords.items():
        if any(term in remarks for term in terms) or func_class_1 in terms:
            return category
    
    return 'Uncategorized'

In [22]:
df['category'] = df.apply(lambda row: classify_individual(row['remarks'], row['func_class_1'], row['func_class_2']), axis=1)

# Group by voyage_code and calculate the total number of people in each category
voyage_grouped = df.groupby('voyage_code').agg(
    total_people=('voyage_code', 'size'),
    total_maritime_leadership=('category', lambda x: (x == 'Maritime Leadership').sum()),
    total_military_leadership=('category', lambda x: (x == 'Military Leadership').sum()),
    total_non_mar_specialized=('category', lambda x: (x == 'Non-mar. Specialized').sum()),
    total_general=('category', lambda x: (x == 'General').sum()),
    total_general_spec=('category', lambda x: (x == 'General Spec').sum()),
    total_foreign_labour=('category', lambda x: (x == 'Foreign labour force').sum()),
    total_servants=('category', lambda x: (x == 'Servant').sum()),
    total_specialized_labour=('category', lambda x: (x == 'Specialized Labour').sum()),
    total_passengers=('category', lambda x: (x == 'Passenger').sum()),
    total_unfree_labour=('category', lambda x: (x == 'Unfree Labour').sum()),
    total_religious=('category', lambda x: (x == 'Religious').sum()),
    total_medical=('category', lambda x: (x == 'Medical').sum()),
    total_company_admin=('category', lambda x: (x == 'Company Admin').sum()),
    total_prisoners=('category', lambda x: (x == 'Prisoner').sum()),
    total_stowaways=('category', lambda x: (x == 'Stowaway').sum()),
    total_other=('category', lambda x: (x == 'Other/No info').sum())
).reset_index()

# Calculate the percentage of each category on each voyage
for col in ['maritime_leadership', 'military_leadership', 'non_mar_specialized', 'general', 'general_spec', 'foreign_labour', 'servants', 'specialized_labour', 'passengers', 'unfree_labour', 'religious', 'medical', 'company_admin', 'prisoners', 'stowaways', 'other']:
    voyage_grouped[f'percentage_{col}'] = (voyage_grouped[f'total_{col}'] / voyage_grouped['total_people']) * 100


In [23]:
df

,file_name,n,age,build,complexion,conscription,wage,remarks,f_n,zot_title,...,alternate_last_name,alternate_first_name,parents,gender,families_groups,is_active_duty,is_passenger,is_servant,is_armurier,category
0,1722 - Rôle du Solide_2028,1.0,NaN,NaN,NaN,NaN,75,a fait la campagne,NaN,Rôle du Solide (1722),...,MANCEL,NaN,NaN,m,NaN,False,False,False,False,Maritime Leadership
1,1722 - Rôle du Solide_2028,6.0,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,Rôle du Solide (1722),...,NaN,NaN,NaN,m,NaN,False,False,False,False,Maritime General
2,1722 - Rôle du Solide_2028,7.0,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,Rôle du Solide (1722),...,NaN,NaN,NaN,m,NaN,False,False,False,False,Maritime General
3,1722 - Rôle du Solide_2028,8.0,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,Rôle du Solide (1722),...,NaN,NaN,NaN,m,NaN,False,False,False,False,Maritime General
4,1722 - Rôle du Solide_2028,3.0,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,Rôle du Solide (1722),...,NaN,NaN,NaN,m,NaN,False,False,False,False,Maritime General
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215659,ROLE2P44-II.9_2047,807.0,NaN,NaN,NaN,NaN,6,complément à l'île de France le 08/08/1770 \n ...,NaN,Rôle du Mars (1770 -,...,NaN,NaN,NaN,m,NaN,False,False,False,False,Maritime General Spec
215660,ROLE2P44-II.9_2047,808.0,NaN,NaN,NaN,NaN,6,complément à l'île de France le 08/08/1770 \n ...,NaN,Rôle du Mars (1770 -,...,NaN,NaN,NaN,m,NaN,False,False,False,False,Maritime General Spec
215661,ROLE2P51-34_1847,101.0,NaN,NaN,NaN,NaN,sans solde,remplacement à l'île de France le 17/12/1785 \...,NaN,Rôle de la Bretagne (1785),...,NaN,NaN,NaN,m,NaN,False,False,False,False,Uncategorized
215662,ROLE2P39-II.9_1645,286.0,NaN,NaN,NaN,NaN,7.10,remplacement à Wampou le 02/12/1763 \n débarqu...,NaN,Rôle du Beaumont (1763 - 1764),...,NaN,NaN,Pierre,m,NaN,True,False,False,False,Military General


In [24]:
voyage_grouped

,voyage_code,total_people,total_maritime_leadership,total_military_leadership,total_non_mar_specialized,total_general,total_general_spec,total_foreign_labour,total_servants,total_specialized_labour,...,percentage_servants,percentage_specialized_labour,percentage_passengers,percentage_unfree_labour,percentage_religious,percentage_medical,percentage_company_admin,percentage_prisoners,percentage_stowaways,percentage_other
0,ACH01,961,26,7,0,0,0,0,2,1,...,0.208117,0.104058,0.000000,0.208117,0.0,0.0,0.0,0.0,0.0,0.0
1,ACH02,736,27,14,0,0,0,0,13,1,...,1.766304,0.135870,0.135870,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,ACH03,582,18,7,0,0,0,0,13,2,...,2.233677,0.343643,14.948454,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,ACH04,729,39,17,0,0,0,0,11,1,...,1.508916,0.137174,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,ACH05,16,4,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,12.500000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,ZEP05,42,6,0,0,0,0,0,0,0,...,0.000000,0.000000,2.380952,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1279,ZEP06,22,3,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1280,ZEP07,33,3,1,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1281,ZEP08,18,3,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
voyage_grouped.iloc[0]

voyage_code                          ACH01
total_people                           961
total_maritime_leadership               26
total_military_leadership                7
total_non_mar_specialized                0
total_general                            0
total_general_spec                       0
total_foreign_labour                     0
total_servants                           2
total_specialized_labour                 1
total_passengers                         0
total_unfree_labour                      2
total_religious                          0
total_medical                            0
total_company_admin                      0
total_prisoners                          0
total_stowaways                          0
total_other                              0
percentage_maritime_leadership    2.705515
percentage_military_leadership    0.728408
percentage_non_mar_specialized         0.0
percentage_general                     0.0
percentage_general_spec                0.0
percentage_